In [ ]:
## Input geneblock
import pandas as pd
import numpy as np
from itertools import combinations 

data=pd.read_csv("problematic antibodies.csv")
H1=data.loc[25:34]
H2=data.loc[49:56]
H3=data.loc[96:109]
L1=data.loc[144:153]
L2=data.loc[171:177]
L3=data.loc[210:218]
Position=data.iloc[0:121,0]
Position_light=data.iloc[122:300,0]

# Genblocks bococizumab
Genblock_heavy_all=['CAAGTTCAATTGGTTCAATCTGGTGCTGAAGTTAAGAAACCAGGTGCTTCAGTTAAAGTTTCTTGTAAAGCTTCTGGTTATACTTTTACTTCTTATTACATGCATTGGGTTAGACAAGCTCCAGGTCAAGGTTTGGAATGGATGGGTGAAATCTCTCCATTTGGTGGTAGAACTAACTATAATGAAAAGTTTAAGTCTAGAGTTACTATGACTAGAGATACTTCTACATCTACTGTATATATGGAATTGTCTTCTTTGAGATCTGAAGATACTGCTGTTTATTACTGTGCTAGAGAAAGACCATTGTATGCTTCTGATTTGTGGGGTCAGGGTACTACTGTTACTGTTTCTTCT',
                    'GAAGTTCAATTAGTTGAATCCGGTGGTGGTTTGGTTCAACCAGGTGGTTCTCTTAGATTGTCTTGCGCTGCTTCTGGTTTTACTTTGTCTGGTGATTGGATTCATTGGGTTAGACAAGCTCCAGGTAAAGGTTTGGAATGGGTTGGTGAAATTAGTGCTGCTGGGGGTTATACTGATTATGCTGATTCTGTTAAAGGTAGATTTACTATTTCTGCTGATACTTCTAAAAACACTGCTTACTTGCAAATGAATTCTTTGAGAGCTGAAGATACTGCTGTCTATTATTGCGCTAGAGAATCTAGAGTTTCTTTTGAAGCTGCTATGGATTATTGGGGTCAAGGTACTTTGGTTACTGTTTCTTCT',
                    'CAAGTTCAATTGGTTCAATCTGGTGCTGAAGTTAAAAAGCCAGGTGCTTCTGTTAAAGTTTCTTGTAAAGCTTCTGGTTATTCTTTTACCAACTACTATATTCATTGGGTTAGACAAGCTCCAGGTCAAAGATTGGAATGGATGGGTTGGATTAATGCTGGTAATGGTAATACAAAGTATTCACAAAAATTCCAAGGAAGAGTTACTATTACGAGAGATACTTCTGCTTCTACTGCTTATATGGAGTTGTCTTCTTTGAGATCTGAAGATACTGCTGTATACTATTGTGTTAGGAGACAAAGGTTTCCATATTACTTTGATTATTGGGGTCAAGGTACTTTGGTTACTGTTTCTTCT',
                    'CAAGTTCAACTTGTTCAATCTGGTGCTGAAGTTAAGAAACCAGGTGCTTCTGTTAAAGTTTCTTGTAAAGCTTCTGGTTATACTTTTACTGATTACTATATGCATTGGGTTAGACAAGCTCCAGGTCAGGGTTTGGAATGGATGGGTAGAGTTAATCCAAATAGAAGAGGAACTACTTATAATCAAAAGTTTGAAGGTCGTGTAACTATGACTACTGATACTTCTACGTCTACTGCTTATATGGAATTGCGTTCTCTTAGATCTGATGATACTGCTGTATATTACTGTGCTAGAGCTAATTGGTTGGACTATTGGGGTCAAGGCACTACCGTTACTGTTTCTTCT']
Genblock_light_all=['GATATTCAAATGACTCAATCTCCATCTTCTTTGTCTGCTTCTGTTGGTGATAGAGTTACTATAACTTGTCGTGCTTCTCAGGGTATTTCTTCTGCGTTGGCTTGGTATCAGCAAAAACCAGGTAAAGCTCCAAAATTGTTGATTTATTCCGCTTCTTATAGATATACTGGTGTTCCATCTCGTTTTTCCGGTTCAGGTTCTGGTACTGATTTTACTTTTACTATTTCTAGTTTGCAACCAGAAGATATTGCTACTTATTACTGTCAACAAAGATACTCTCTTTGGAGAACTTTTGGTCAAGGTACTAAATTGGAAATTAAG',
                    'GATATTCAAATGACTCAATCTCCATCATCTTTGTCTGCTTCTGTTGGTGATAGAGTTACTATTACTTGTAGAGCTTCTCAAAATATCGCTACTGATGTTGCTTGGTATCAACAAAAACCAGGTAAAGCTCCAAAACTATTGATCTATTCTGCATCTTTCTTGTATTCTGGTGTTCCAAGTAGATTTTCAGGTAGCGGCTCTGGCACGGACTTCACTTTGACTATTTCTTCTTTGCAACCAGAGGATTTTGCTACATATTACTGTCAGCAATCTGAACCAGAACCATATACTTTTGGTCAAGGTACTAAAGTTGAAATTAAG',
                    'GAAATTGTTTTGACTCAGTCTCCGGCCACTTTGTCTGTTTCTCCAGGTGAAAGAGCTACGTTGTCTTGTAGAGCTTCTCAATCTGTTGGGACTAATGTTGCTTGGTATCAACAAAAACCAGGTCAAGCTCCAAGAGTTTTGATATATTCTACTTCTTCTAGAGCCACTGGTATTACTGATAGATTTTCAGGTTCTGGATCTGGTACTGATTTTACTTTGACTATTTCTAGATTGGAACCCGAAGATTTTGCTGTTTACTATTGTCAACAGTTTAACAAATCTCCCTTGACTTTTGGTGGTGGTACTAAAGTTGAAATTAAG',
                    'GATATTCAAATGACTCAATCTCCATCTTCTTTGTCTGCTAGTGTTGGTGATAGAGTTACTATTACTTGTTCTGTTTCCTCTTCCGTTTCTTCTATTTACTTGCATTGGTATCAACAAAAACCAGGTAAAGCTCCAAAACTTTTGATTTATTCTACTTCTAATTTGGCTTCTGGAGTTCCCTCTAGATTTTCCGGTTCTGGAAGCGGGACTGATTTTACTTTGACTATTTCTTCCCTACAACCAGAAGATTTTGCTACTTATTACTGCCAAGTTTACTCCGGTTATCCATTGACTTTTGGTGGTGGTACTAAAGTTGAAATTAAG']
#len(Genblock)
# which antibody
# bococizumab:1, duligotuzumab:2, lenzilumab:3, emibutuzumab:4
n2=3
n1=n2+4
Heavy_f1=data.iloc[0:121,n2+4]
Light_f1=data.iloc[121:300,n2+4]
Genblock=Genblock_heavy_all[n2-1]
Genblock_light=Genblock_light_all[n2-1]

In [ ]:
## Input definition and constants
# generate a codon table to map the amino acid
def translate(seq): 
       
    table = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W', 
    } 
    protein ="" 
    if len(seq)%3 == 0: 
        for i in range(0, len(seq), 3): 
            codon = seq[i:i + 3] 
            protein+= table[codon] 
    return protein

def Symbol3(cod):
    
    table={'CGT':'B','CTG':'B','GTC':'B','GCT':'B','TCG':'B','TGC':'B',
           'AGT':'D','ATG':'D','TAG':'D','TGA':'D','GTA':'D','GAT':'D',
           'ACT':'H','ATC':'H','TAC':'H','TCA':'H','CTA':'H','CAT':'H',
           'ACG':'V','AGC':'V','CGA':'V','CAG':'V','GAC':'V','GCA':'V'}
    protein=""
    if len(cod)%3 ==0:
        for i in range(0, len(cod), 3): 
            codon = cod[i:i + 3] 
            protein+= table[codon] 
    return protein

def Symbol2(cod):
    
    table={'AT':'W','CG':'S','AC':'M','GT':'K','AG':'R','CT':'Y',
          'TA':'W','GC':'S','CA':'M','TG':'K','GA':'R','TC':'Y'}
    protein=""
    if len(cod)%2 ==0:
        for i in range(0, len(cod), 2): 
            codon = cod[i:i + 2] 
            protein+= table[codon] 
    return protein

#based on the designed library
def DeGCod(AA):
    
    table={'R':'RVR','Y':'KHY','W':'KBG','A':'KHY','L':'VWA','G':'RVY','P':'SHR','F':'KHY','V':'SHR',
          'H':'-','I':'RHA','K':'RHA','M':'-'}
    protein=""
    if len(AA)%1==0:
        for i in range(0,len(AA),1):
            codon = AA[i:i+1]
            protein+=table[AA]
    return protein
def DeGCod_comp(AA):
    
    table={'R':'YBY','Y':'MDR','W':'MVC','A':'MDR','L':'BWT','G':'YBR','P':'SDY','F':'MDR','V':'SDY',
          'H':'-','I':'YDT','K':'YDT','M':'-'}
    protein=""
    if len(AA)%1==0:
        for i in range(0,len(AA),1):
            codon = AA[i:i+1]
            protein+=table[AA]
    return protein

# generate complementary chain
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)
    def reverse_complement(s):
        return complement(s[::-1])
    
# reverse chain from 5'-3'
def reverse(s): 
    if len(s) == 0: 
        return s 
    else: 
        return reverse(s[1:]) + s[0] 
    



In [ ]:
#limit for natural diversity of 50%
x=np.array([["HCDR1","H26","G"],
           ["HCDR1","H29","F"],
           ["HCDR1","H32","Y"],
           ["HCDR2","H51","I"],
           ["HCDR2","H55","G"],
           ["HCDR3","H93","A"],
           ["HCDR3","H94","R"]])
NDAA=pd.DataFrame(x,columns=['Region','Position','AA'])
y=np.array([["LCDR1","L32","Y"],
           ["LCDR1","L33","L"],
           ["LCDR2","L51","A"],
           ["LCDR2","L54","R"]])
NDAA_light=pd.DataFrame(y,columns=['Region','Position','AA'])
NDAA.shape

In [ ]:
## heavy chain libraries: identify sites and amino aicd --Step1 
df_H1=H1.iloc[:,n1]
SASA_H1=H1.iloc[:,n2]
df_H2=H2.iloc[:,n1]
SASA_H2=H2.iloc[:,n2]
df_H3=H3.iloc[:,n1]
SASA_H3=H3.iloc[:,n2]
position=[]
s=0.1
s=0
for i in range(0,len(df_H1)):
    if df_H1.iloc[i]=="G" or df_H1.iloc[i]=="A" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="F" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="H" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="I" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="K" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="L" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="M" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="P" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="R" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="V" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="W" and SASA_H1.iloc[i]>=s or df_H1.iloc[i]=="Y" and SASA_H1.iloc[i]>=s:
        position.append(("HCDR1",H1.iloc[i,0],df_H1.iloc[i],SASA_H1.iloc[i],DeGCod(df_H1.iloc[i]),DeGCod_comp(df_H1.iloc[i])))

for i in range(0,len(df_H2)):
    if df_H2.iloc[i]=="G" or df_H2.iloc[i]=="A" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="F" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="H" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="I" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="K" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="L" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="M" and SASA_H2.iloc[i]>s or df_H2.iloc[i]=="P" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="R" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="V" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="W" and SASA_H2.iloc[i]>=s or df_H2.iloc[i]=="Y" and SASA_H2.iloc[i]>=s:
        position.append(("HCDR2",H2.iloc[i,0],df_H2.iloc[i],SASA_H2.iloc[i],DeGCod(df_H2.iloc[i]),DeGCod_comp(df_H2.iloc[i])))
        
for i in range(0,len(df_H3)):
    if df_H3.iloc[i]=="G" or df_H3.iloc[i]=="A" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="F" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="H" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="I" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="K" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="L" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="M" and SASA_H3.iloc[i]>s or df_H3.iloc[i]=="P" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="R" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="V" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="W" and SASA_H3.iloc[i]>=s or df_H3.iloc[i]=="Y" and SASA_H3.iloc[i]>=s:
        position.append(("HCDR3",H3.iloc[i,0],df_H3.iloc[i],SASA_H3.iloc[i],DeGCod(df_H3.iloc[i]),DeGCod_comp(df_H3.iloc[i])))
        
position
position_df=pd.DataFrame(position,columns=['Region','Position','Original AA','SASA score','De codon','De codon comp'])  

In [ ]:
## Heavy chain libraries: eliminate sites with max ND--Step2
row=[]
for i in range (position_df.shape[0]):
    for j in range(NDAA.shape[0]):
        if position_df.iloc[i,0]==NDAA.iloc[j,0] and position_df.iloc[i,1]==NDAA.iloc[j,1] and position_df.iloc[i,2]==NDAA.iloc[j,2]:
            #print(position_df.iloc[i,1])
            row.append(i)
position_drop=position_df.drop(row)
position_drop.sort_values(by=['Region'], ascending=True)
print len(position_drop)
print position_drop

In [ ]:
## Heavy chain: change degenerate codon--Step3 
mapcod=[]
mapcod=pd.DataFrame({'Map codon':[]})
mapcod_ori=pd.DataFrame({'Map codon origin':[]})
mapcod_c=pd.DataFrame({'Map codon comp':[]})
mapcod_c_ori=pd.DataFrame({'Map codon comp origin':[]})
for i in range (0,len(Genblock),3):
    mapcod=mapcod.append({'Map codon':Genblock[i:i+3]}, ignore_index=True)
    mapcod_ori=mapcod_ori.append({'Map codon origin':Genblock[i:i+3]}, ignore_index=True)
    mapcod_c=mapcod_c.append({'Map codon comp':complement(Genblock[i:i+3])}, ignore_index=True)
    mapcod_c_ori=mapcod_c_ori.append({'Map codon comp origin':complement(Genblock[i:i+3])}, ignore_index=True)
    
Heavy_f1_modi = pd.concat([Position, Heavy_f1], axis=1, sort=False)
Heavy_f1_new=Heavy_f1_modi.drop(Heavy_f1_modi[Heavy_f1_modi.iloc[:,1]=='-'].index,axis=0)
Heavy_f1_new2 = pd.DataFrame([Heavy_f1_new.iloc[:,0].values,Heavy_f1_new.iloc[:,1].values]).T
Heavy_frame = pd.concat([Heavy_f1_new2,mapcod, mapcod_ori,mapcod_c, mapcod_c_ori], axis=1, sort=False)

List=Heavy_frame.iloc[:,0]
unique_index = pd.Index(List)
for i in range(len(position_drop)):
    index=unique_index.get_loc(position_drop.iloc[i,1])
    Heavy_frame.iloc[index,2]=position_drop.iloc[i,4] 
    Heavy_frame.iloc[index,4]=position_drop.iloc[i,5] 
# calculate primer's starting position and end position
unique_index = pd.Index(Heavy_frame.iloc[:,0])
l1s=unique_index.get_loc('H26')
l1e=unique_index.get_loc('H35')
l2s=unique_index.get_loc('H50')
l2e=unique_index.get_loc('H65')
l3s=unique_index.get_loc('H93')
l3e=unique_index.get_loc('H102')
print l1s,l1e,l2s,l2e,l3s,l3e
#Light_frame.head(10)

In [ ]:
# Plan A primer design
#forward 3rd primer
F3_1f=''
F3_1r=''
# generate a broader region for H1 region
f3_1f=Heavy_frame.iloc[l1s-8:l1e+1+14,2]
f3_1r=Heavy_frame.iloc[l1s-8:l1e+1+14,4]
for i in range (len(f3_1f)):
    F3_1f=F3_1f+f3_1f.iloc[i]
    F3_1r=F3_1r+f3_1r.iloc[i]

# not paired sequence after H1
F3_3f=''
F3_3r=''
f3_3f=Heavy_frame.iloc[l1e+7:l1e+14,2]
f3_3r=Heavy_frame.iloc[l1e+7:l1e+14,4]
for i in range (len(f3_3f)):
    F3_3f=F3_3f+f3_3f.iloc[i]
    F3_3r=F3_3r+f3_3r.iloc[i]

# sequence for H1 region
F3_2f=''
F3_2r=''
f3_2f=Heavy_frame.iloc[l1s:l1e+1,2]
f3_2r=Heavy_frame.iloc[l1s:l1e+1,4]
for i in range (len(f3_2f)):
    F3_2f=F3_2f+f3_2f.iloc[i]
    F3_2r=F3_2r+f3_2r.iloc[i]
    
F3_H1startf=F3_1f.find(F3_2f)
F3_1f.rfind('G')

# 2nd Primer, reverse sequence before H1 region
F2_1r=F3_1r[min(F3_1f.index('C'),F3_1f.index('G')):F3_H1startf]
Tm_1 = 4*(F2_1r.count('C')+F2_1r.count('G'))+2*(F2_1r.count('A')+F2_1r.count('T'))
Tm_2 = 4*(F3_3f.count('C')+F3_3f.count('G'))+2*(F3_3f.count('A')+F3_3f.count('T'))
# 
print Tm_1,Tm_2,len(F3_1f),len(F3_3f)
print F3_2f, len(F3_2f)

# sequence for H2 region (reverse)
F4_2f=''
F4_2r=''
f4_2f=Heavy_frame.iloc[l2s:l2e+1,2]
f4_2r=Heavy_frame.iloc[l2s:l2e+1,4]
for i in range (len(f4_2f)):
    F4_2f=F4_2f+f4_2f.iloc[i]
    F4_2r=F4_2r+f4_2r.iloc[i]
    
F4_2r=reverse(F4_2r[0:len(F4_2r)])
print F4_2r, len(F4_2r)

# sequence for H3 region (reverse)
F6_2f=''
F6_2r=''
f6_2f=Heavy_frame.iloc[l3s:l3e+1,2]
f6_2r=Heavy_frame.iloc[l3s:l3e+1,4]
for i in range (len(f6_2f)):
    F6_2f=F6_2f+f6_2f.iloc[i]
    F6_2r=F6_2r+f6_2r.iloc[i]
    
F6_2r=reverse(F6_2r[0:len(F6_2r)])
print F6_2r,len(F6_2r)